In [ ]:
import pandas as pd
df = pd.read_excel('TEJ.xlsx', sheet_name='result_FINAL') #sheet_name 決定資料

In [ ]:
import os
files = list(map(lambda x: x.split('.txt')[0], filter(lambda x: '.txt' in x, os.listdir('致股東報告書')))) #讀取檔名

In [ ]:
import math
result = []
remove_list = ['【', '】', '「', '｣', '（', '）', '［', '］', '〖', '〗', '『', '』', '。', '，', '、', '：', '；', '～', '！',
          '＠', '＃', '％', '＾', '＆', '＝', '＋', '－', '＊', '／', '＼', '＿', '？', '〈', '〉', '《', '》', '[', ']', 
          '{', '}', '<', '>', '(', ')', '‘', '’', '.', ',', ':', ';', '`', '~', '!', '@', '#', '%', '%', '^', '&', 
          '_', '=', '+', '-', '*', '/', '\\', '_', '?', '★', '◎', '\\n', '\\t', '元', 'cid', '\\uf06a', '致股東報告書', 
          '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '壹', '貳', '參', '肆', '伍', '陸', '柒', '捌', '玖', '拾',
          '佰', '仟', '萬', '億', '兆', '零', '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', '○']

for index, data in df.iterrows():
    file_name = data['檔名']
    value = data['y_營業收入淨額']
    
    if not math.isnan(value) and file_name in files:
        with open('致股東報告書/{0}.txt'.format(file_name) ,encoding="utf-8") as f:
            text = f.read()
            for r in remove_list:
                text = text.replace(r, '')
            text = text.strip(' ')
            text = ' '.join(text.split())
            if len(text) > 50:
                result.append({'file_name':file_name , 'text': text, 'result': '0' if value == 0 else '1'})

In [ ]:
#result[100]

In [ ]:
len(result)

In [ ]:
import numpy as np
file_name = [x['file_name'] for x in result]
docs = [x['text'] for x in result]
y_data = np.array([x['result'] for x in result])

In [ ]:
import jieba 
jieba.set_dictionary('./dict.txt.big.txt')

def token(text):
    texts=jieba.cut(text)
    text=[]
    for j in texts:
        text.append(j)
    return text

In [ ]:
#token後的所有文章
texts=[] 
for i in range(len(docs)):
    temp = token(docs[i])
    temp = list(filter(lambda x: x != ' ', temp))
    texts.append(temp)

In [ ]:
docs_final=[] #改成可以直接丟進去CountVectorizer套件的格式
for i in range(len(texts)):
    a=" ".join(texts[i])
    docs_final.append(a)
#docs_final[100]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
TF_vectorizer = CountVectorizer(stop_words='english')
TF_vectors = TF_vectorizer.fit_transform(docs_final)
TF_vectors.shape

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda_model = LDA(n_components = 768, random_state = 87)
LDA_vectors = lda_model.fit_transform(TF_vectors)
LDA_vectors.shape

In [ ]:
import random
random.seed(87)
random.shuffle(LDA_vectors)
random.seed(87)
random.shuffle(y_data)
random.seed(87)
random.shuffle(file_name)

In [ ]:
data_lda = {name: {'x': LDA_vectors[index], 'y': y_data[index]} for index, name in enumerate(file_name)}

In [ ]:
import pickle
file = open("./LDA/營業收入淨額_lda_data.pkl", "wb")
pickle.dump(data_lda, file)
file.close()